- now that you've created a catkin workspace and added the `simple_arm` package to it, you're ready to write some ROS nodes
- in this lesson, you'll be writing nodes in Python that publish and subscribe to topics and you'll write a ROS service that can be called from other nodes or from the command line


- the first node that you will be writing is called `simple_mover` which does nothing more than publish joint angle commands to `simple_arm`


- after you've developed a basic understanding of the general structure of a ROS Node written in Python, you will be writing another node called `arm_mover` which provides a service called `safe_move`, which allows the arm to be moved to any position within its workspace which has been deemed to be “safe”
- the safe zone is bounded by minimum and maximum joint angles, and is configurable via the ROS’ parameter server


- the last node you’ll write in this lesson is the `look_away` node which subscribes to a topic where camera data is being published
- when the camera detects an image with uniform color, meaning it’s looking at the sky, the node will call the `safe_move` service to move the arm to a new position

# ROS Publishers

- before you see the code for `simple_mover`, it may be helpful to see how ROS Publishers work in Python
- publishers allow a node to send messages to a topic, so that data from the node can be used in other parts of the ROS system
- in Python, ROS publishers typically have the following definition format, although other parameters and arguments are possible:

```python
pub1 = rospy.Publisher("/topic_name", message_type, queue_size=size)
```


- ROS publishing can be either synchronous or asynchronous:
  - synchronous publishing means that a publisher will attempt to publish to a topic but may be blocked if that topic is being published to by a different publisher
    - in this situation, the second publisher is blocked until the first publisher has serialized all messages to a buffer and the buffer has written the messages to each of the topic's subscribers
    - this is the default behavior of a `rospy.Publisher` if the `queue_size` parameter is not used or set to `None`
  - asynchronous publishing means that a publisher can store messages in a queue until the messages can be sent
    - if the number of messages published exceeds the size of the queue, the oldest messages are dropped
    - the queue size can be set using the `queue_size` parameter


- once the publisher has been created as above, a message with the specified data type can be published as follows:

```python
pub1.publish(message)
```


- for more information about ROS publishers, see the documentation [here](http://docs.ros.org/kinetic/api/rospy/html/rospy.topics.Publisher-class.html)

- **Q:** Assume that a queued message is typically picked up in an average time of 1/10th of a second with a standard deviation of 1/20th of a second, and your publisher is publishing at a frequency of 10Hz. Of the options below, which would be the best setting for queue_size?
- **A:** Choosing a good queue_size is somewhat subjective, but since messages are picked up at roughly the same rate they are published, a queue_size of 2 provides a little room for messages to queue without being too large.

# Simple Mover

- you will now go through the process of implementing your first ROS node in python
- this node is called `simple_mover` and as it’s name implies, this node only has one responsibility, and that is to command joint movements for `simple_arm`
- to do so, it must publish joint angle command messages to the following topics:
  - topic name: /simple_arm/joint_1_position_controller/command
    - message type: std_msgs/Float64
    - description: Commands joint 1 to move counter-clockwise, units in radians
  - topic name: /simple_arm/joint_2_position_controller/command
    - message type: std_msgs/Float64
    - description: Commands joint 2 to move counter-clockwise, units in radians


#### Adding the scripts directory

- in order to create a new node in python, you must first create the `scripts` directory within the `simple_arm` package, as it does not yet exist

```shell
cd ~/catkin_ws/src/simple_arm/
mkdir scripts
```


#### Creating a new script

- once the scripts directory has been created, executable scripts can be added to the package
- however, in order for `rosrun` to find them, their permissions must be changed to allow execution
- let’s add a simple bash script that prints “Hello World” to the console

```shell
cd scripts
echo '#!/bin/bash' >> hello
echo 'echo Hello World' >> hello
```

- after setting the appropriate execution permissions on the file, rebuilding the workspace, and sourcing the newly created environment, you will be able to run the script

```shell
chmod u+x hello
cd ~/catkin_ws
catkin_make
source devel/setup.bash
rosrun simple_arm hello
```


#### Creating the empty simple_mover node script

- to create the `simple_mover` node script, you will must simply follow the same basic routine introduced a moment ago

```shell
cd ~/catkin_ws/src/simple_arm
cd scripts
touch simple_mover
chmod u+x simple_mover
```

- you can now edit the empty `simple_mover` script with your favorite text editor

# Simple Mover: The Code

- below is the complete code for the `simple_mover` node, in it’s entirety, followed by a step-by-step explanation of what is happening

```python
#!/usr/bin/env python

import math
import rospy
from std_msgs.msg import Float64

def mover():
    pub_j1 = rospy.Publisher('/simple_arm/joint_1_position_controller/command',
                             Float64, queue_size=10)
    pub_j2 = rospy.Publisher('/simple_arm/joint_2_position_controller/command',
                             Float64, queue_size=10)
    rospy.init_node('arm_mover')
    rate = rospy.Rate(10)
    start_time = 0

    while not start_time:
        start_time = rospy.Time.now().to_sec()

    while not rospy.is_shutdown():
        elapsed = rospy.Time.now().to_sec() - start_time
        pub_j1.publish(math.sin(2*math.pi*0.1*elapsed)*(math.pi/2))
        pub_j2.publish(math.sin(2*math.pi*0.1*elapsed)*(math.pi/2))
        rate.sleep()

if __name__ == '__main__':
    try:
        mover()
    except rospy.ROSInterruptException:
        pass

```

### The code: Explained

```python
#!/usr/bin/env python

import math
import rospy
```

- `rospy` is the official Python client library for ROS
- it provides most of the fundamental functionality required for interfacing with ROS via Python
- it has interfaces for creating Nodes, interfacing with Topics, Services, Parameters, and more
- it will certainly be worth your time to check out the API documentation [here](http://docs.ros.org/kinetic/api/rospy/html/)
- general information about rospy, including other tutorials may be found on the [ROS Wiki](http://wiki.ros.org/rospy_tutorials/Tutorials/WritingPublisherSubscriber)

```python
from std_msgs.msg import Float64
```

- from the `std_msgs` package, we import `Float64`, which is one of the primitive message types in ROS
- the [std_msgs](http://wiki.ros.org/std_msgs) package also contains all of the other primitive types
- later on in this script, we will be publishing `Float64` messages to the position command topics for each joint

```python
def mover():
    pub_j1 = rospy.Publisher('/simple_arm/joint_1_position_controller/command', Float64, queue_size= 10)
    pub_j2 = rospy.Publisher('/simple_arm/joint_2_position_controller/command', Float64, queue_size=10)
```

- at the top of the `mover` function, two publishers are declared, one for joint 1 commands, and one for joint 2 commands
- here, the `queue_size` parameter is used to determine the maximum number messages that may be stored in the publisher queue before messages are dropped
- more information about this parameter can be found [here](http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers#queue_size:_publish.28.29_behavior_and_queuing)

```python
rospy.init_node('arm_mover')
```

- initializes a client node and registers it with the master
- here “arm_mover” is the name of the node
- `init_node()` must be called before any other rospy package functions are called
- the argument `anonymous=True` makes sure that you always have a unique name for your node

```python
rate = rospy.Rate(10)
```

- the `rate` object is created here with a value of 10 Hertz
- rates are used to limit the frequency at which certain loops spin in ROS
- choosing a rate which is too high may result in unnecessarily high CPU usage, while choosing a value too low could result in high overall system latency
- choosing sensible values for all of the nodes in a ROS system is a bit of a fine-art

```python
start_time = 0

while not start_time:
    start_time = rospy.Time.now().to_sec()
```

- `start_time` is used to determine how much time has elapsed
- when using ROS with simulated time (as we are doing here), `rospy.Time.now()` will initially return 0, until the first message has been received on the` /clock` topic
- this is why `start_time` is set and polled continuously until a nonzero value is returned (more information [here](http://wiki.ros.org/rospy/Overview/Time))

```python
while not rospy.is_shutdown():
    elapsed = rospy.Time.now().to_sec() - start_time
    pub_j1.publish(math.sin(2*math.pi*0.1*elapsed)*(math.pi/2))
    pub_j2.publish(math.sin(2*math.pi*0.1*elapsed)*(math.pi/2))
    rate.sleep()
```

- this the main loop
- due to the call to `rate.sleep()`, the loop is traversed at approximately 10 Hertz
- each trip through the body of the loop will result in two joint command messages being published
- the joint angles are sampled from a sine wave with a period of 10 seconds, and in magnitude from $[-\pi/2, +\pi/2]$
- when the node receives the signal to shut down (either from the master, or via SIGINT signal in a console window), the loop will be exited

```python
if __name__ == '__main__':
    try:
        mover()
    except rospy.ROSInterruptException:
        pass
```

- if the name variable is set to `“main”`, indicating that this script is being executed directly, `he `mover()` function will be called
- the try/except blocks here are significant as rospy uses exceptions extensively
- the particular exception being caught here is the `ROSInterruptException`
- this exception is raised when the node has been signaled for shutdown
- if there was perhaps some sort of cleanup needing to be done before the node shuts down, it would be done here
- more information about rospy exceptions can be found [here](http://wiki.ros.org/rospy/Overview/Exceptions)

### Running simple_mover

- assuming that your workspace has recently been built, and it’s `setup.bash` has been sourced, you can launch simple_arm as follows:

```shell
cd ~/catkin_ws
roslaunch simple_arm robot_spawn.launch
```

- once ROS Master, Gazebo, and all of our relevant nodes are up and running, we can finally launch `simple_mover`
- to do so, open a new terminal and type the following commands:

```shell
cd ~/catkin_ws
source devel/setup.bash
rosrun simple_arm simple_mover
```

# ROS Services

- now that you've written your first ROS node, you've seen how publishing to a topic works, and you were able control the robotic arm by publishing to the `/simple_arm/joint_2_position_controller/command` topic
- next up, we'll see another node called `arm_mover` which implements the `safe_move` service to allow the arm to be controlled with service calls


### Defining services

- a ROS service allows request/response communication to exist between nodes
- within the node providing the service, request messages are handled by functions or methods
- once the requests have been handled successfully, the node providing the service sends a message back to the requester node
- in Python, a ROS service can be created using the following definition format:
```python
service = rospy.Service('service_name', serviceClassName, handler)
```
- here, the `service_name` is the name given to the service
  - other nodes will use this name to specify which service they are sending requests to
- the `serviceClassName` comes from the file name where the service definition exists
  - you will see more about this in the next classroom concept, but each service has a definition provided in an `.srv` file; this is a text file that provides the proper message type for both requests and responses
- the `handler` is the name of the function or method that handles the incoming service message
  - this function is called each time the service is called, and the message from the service call is passed to the `handler` as an argument
  - the `handler` should return an appropriate service response message


### Using Services

- services can be called directly from the command line, and you will see an example of this in the upcoming `arm_mover` classroom concepts
- on the other hand, to use a ROS service from within another node, you will define a `ServiceProxy`, which provides the interface for sending messages to the service:

```python
service_proxy = rospy.ServiceProxy('service_name', serviceClassName)
```

- one way the ServiceProxy can then be used to send requests is as follows:

```python
msg = serviceClassNameRequest()
#update msg attributes here to have correct data
response = service_proxy(msg)
```

- in the code above, a new service message is created by calling the `serviceClassNameRequest()` method
  - this method is provided by rospy, and its name is given by appending `Request()` to the name used for `serviceClassName`
- since the message is new, the message attributes should be updated to have the appropriate data
- next, the `service_proxy` can be called with the message, and the response stored


- for other ways to pass data to `service_proxy`, see the ROS documentation [here](http://wiki.ros.org/rospy/Overview/Services)

- **Q:** Which of the following ROS nodes might best be implemented using a service?
- **A:** A kill request is something that might work well as a service. A node that sets parameters on request is a perfect example of a service. In fact, this service exists in turtlesim under turtleX/set_pen. A node which executes movement for a robotic arm is also a good example of a service. In fact, this is the service you will be implementing next!

# Arm Mover

- you’ve written your first ROS node!
- this was no trivial task - you’ve had to learn quite a few things to get to this point
- before you will be prepared for the final project, we have some more ground to cover
- namely, we still need to cover:
  - custom message generation
  - services
  - parameters
  - launch Files
  - subscribers
  - logging
- in order to gain an understanding of some of the above, you will be writing another node called `arm_mover`


#### Description of Arm Mover

- in many respects, `arm_mover` is quite similar to `simple_mover`
- like `simple_mover`, it is responsible for commanding the arm to move
- however, instead of simply commanding the arm to follow a predetermined trajectory, the `arm_mover` node provides the service `move_arm`, which allows other nodes in the system to send `movement_commands`


- in addition to allowing movements via a service interface, `arm_mover` also allows for configurable minimum and maximum joint angles, by using parameters

### Creating a new service definition

- as you learned earlier, an interaction with a service consists of two messages being passed
- a request passed to the service, and a response received from the service
- the definitions of the request and response message type are contained within `.srv` files living in the `srv` directory under the package’s root


- let’s define a new service for `simple_arm` - we shall call it `GoToPosition`

```shell
cd ~/catkin_ws/src/simple_arm/
mkdir srv
cd srv
touch GoToPosition.srv
```

- you should now edit `GoToPosition.srv`, so it contains the following:

```shell
float64 joint_1
float64 joint_2
---
duration time_elapsed
```

- service definitions always contain two sections, separated by a ‘---’ line
  - the first section is the definition of the request message
    - here, a request consists of two float64 fields, one for each of `simple_arm`’s joints
  - the second section contains is the service response
    - the response contains only a single field, `time_elapsed` which is of type duration, and is responsible for indicating how long it took the arm to perform the movement


- defining a custom message type is very similar, with the only differences being that message definitions live within the `msg` directory of the package root, have a `.msg` extension, rather than `.srv`, and do not contain the “---” section divider
- you can find more detailed information on creating messages and services [here](http://wiki.ros.org/msg), and [here](http://wiki.ros.org/srv), respectively

### Modifying CMakeLists.txt

- in order for catkin to generate the python modules or C++ libraries which allow you to utilize messages in your code you must first modify `simple_arm`’s `CMakeLists.txt` (`~/catkin_ws/src/simple_arm/CMakeLists.txt`)
- CMake is the build tool underlying catkin, and `CMakeLists.txt` is nothing more than a CMake script used by catkin


- first, ensure that the `find_package()` macro lists `std_msgs` and `message_generation` as required packages
- the `find_package()` macro should look as follows:

```shell
find_package(catkin REQUIRED COMPONENTS
        std_msgs
        message_generation
)
```

- as the names might imply, the `std_msgs` package contains all of the basic message types, and `message_generation` is required to generate message libraries for all the supported languages (cpp, lisp, python, javascript)


- in your `CMakeLists.txt`, you may also see `controller_manager` listed as a required package
- in actuality this package is not required
- it was simply added as a means to demonstrate a build failure in the previous lesson
- you may remove it from the list of REQUIRED COMPONENTS if you choose


- next, uncomment the commented-out `add_service_files()` macro so it looks like this:

```shell
## Generate services in the 'srv' folder
add_service_files(
   FILES
   GoToPosition.srv
)
```

- this tells catkin which files to generate code for


- lastly, make sure that the `generate_messages()` macro is uncommented, as follows:

```shell
generate_messages(
   DEPENDENCIES
   std_msgs  # Or other packages containing msgs
)
```

- it is this macro that is actually responsible for generating the code


- for more information about CMakeLists.txt check out [this page](http://wiki.ros.org/catkin/CMakeLists.txt) on the ROS wiki

### Modifying package.xml

- now that the `CMakeLists.txt` file has been covered, you should technically be able to build the project
- however, there’s one more file which needs to be modified, `package.xml`


- `package.xml` is responsible for defining many of the package’s properties, such as the name of the package, version numbers, authors, maintainers, and dependencies


- right now, we’re worried about the dependencies
- in the previous lesson you learned about build-time dependencies and run-time package dependencies
- when `rosdep` is searching for these dependencies, it’s the `package.xml` file that is being parsed
- let’s add the `message_generation` and `message_runtime` dependencies

```xml
<buildtool_depend>catkin</buildtool_depend>
<build_depend>message_generation</build_depend>

<run_depend>controller_manager</run_depend>
<run_depend>effort_controllers</run_depend>
<run_depend>gazebo_plugins</run_depend>
<run_depend>gazebo_ros</run_depend>
<run_depend>gazebo_ros_control</run_depend>
<run_depend>joint_state_controller</run_depend>
<run_depend>joint_state_publisher</run_depend>
<run_depend>robot_state_publisher</run_depend>
<run_depend>message_runtime</run_depend>
<run_depend>xacro</run_depend>
```

- you are now ready to build the package!
- for more information about `package.xml`, check out the [ROS Wiki](http://wiki.ros.org/catkin/package.xml)

### Building the package

- if you build the workspace successfully, you should now find that a python package containing a module for the new service `GoToPosition` has been created deep down in the devel directory

```shell
cd ~/catkin_ws
catkin_make
cd devel/lib/python2.7/dist-packages
ls
```


- after sourcing the newly created `setup.bash`, the new `simple_arm` package has now become part of your` PYTHONPATH` environment variable, and is ready for use!

```shell
env | grep PYTHONPATH
```

### Creating the empty arm_mover node script

- the steps you take to create the `arm_mover` node are exactly the same as the steps you took to create the `simple_mover` script, excepting the actual name of the script itself

```shell
cd ~/catkin_ws
cd src/simple_arm/scripts
touch arm_mover
chmod u+x arm_mover
```

- you can now edit the empty arm_mover script with your favorite text editor

# Arm Mover: The Code

```python
#!/usr/bin/env python

import math
import rospy
from std_msgs.msg import Float64
from sensor_msgs.msg import JointState
from simple_arm.srv import *

def at_goal(pos_j1, goal_j1, pos_j2, goal_j2):
    tolerance = .05
    result = abs(pos_j1 - goal_j1) <= abs(tolerance)
    result = result and abs(pos_j2 - goal_j2) <= abs(tolerance)
    return result

def clamp_at_boundaries(requested_j1, requested_j2):
    clamped_j1 = requested_j1
    clamped_j2 = requested_j2

    min_j1 = rospy.get_param('~min_joint_1_angle', 0)
    max_j1 = rospy.get_param('~max_joint_1_angle', 2*math.pi)
    min_j2 = rospy.get_param('~min_joint_2_angle', 0)
    max_j2 = rospy.get_param('~max_joint_2_angle', 2*math.pi)

    if not min_j1 <= requested_j1 <= max_j1:
        clamped_j1 = min(max(requested_j1, min_j1), max_j1)
        rospy.logwarn('j1 is out of bounds, valid range (%s,%s), clamping to: %s',
                      min_j1, max_j1, clamped_j1)

    if not min_j2 <= requested_j2 <= max_j2:
        clamped_j2 = min(max(requested_j2, min_j2), max_j2)
        rospy.logwarn('j2 is out of bounds, valid range (%s,%s), clamping to: %s',
                      min_j2, max_j2, clamped_j2)

    return clamped_j1, clamped_j2

def move_arm(pos_j1, pos_j2):
    time_elapsed = rospy.Time.now()
    j1_publisher.publish(pos_j1)
    j2_publisher.publish(pos_j2)

    while True:
        joint_state = rospy.wait_for_message('/simple_arm/joint_states', JointState)
        if at_goal(joint_state.position[0], pos_j1, joint_state.position[1], pos_j2):
            time_elapsed = joint_state.header.stamp - time_elapsed
            break

    return time_elapsed

def handle_safe_move_request(req):
    rospy.loginfo('GoToPositionRequest Received - j1:%s, j2:%s',
                   req.joint_1, req.joint_2)
    clamp_j1, clamp_j2 = clamp_at_boundaries(req.joint_1, req.joint_2)
    time_elapsed = move_arm(clamp_j1, clamp_j2)

    return GoToPositionResponse(time_elapsed)

def mover_service():
    rospy.init_node('arm_mover')
    service = rospy.Service('~safe_move', GoToPosition, handle_safe_move_request)
    rospy.spin()

if __name__ == '__main__':
    j1_publisher = rospy.Publisher('/simple_arm/joint_1_position_controller/command',
                                   Float64, queue_size=10)
    j2_publisher = rospy.Publisher('/simple_arm/joint_2_position_controller/command',
                                   Float64, queue_size=10)

    try:
        mover_service()
    except rospy.ROSInterruptException:
        pass
```

### The code: explained

```python
#!/usr/bin/env python

import math
import rospy
from std_msgs.msg import Float64
from sensor_msgs.msg import JointState
from simple_arm.srv import *
```

- the imported modules for `arm_mover` are the same as `simple_arm`, with the exception of two new imports:
  - `JointState messages` are published to the `/simple_arm/joint_states` topic, and are used for monitoring the position of the arm
  - the `simple_arm` package, and the `srv` module are automatically generated by catkin as part of the build process


```python
def at_goal(pos_j1, goal_j1, pos_j2, goal_j2):
    tolerance = .05
    result = abs(pos_j1 - goal_j1) <= abs(tolerance)
    result = result and abs(pos_j2 - goal_j2) <= abs(tolerance)
    return result
```

- this function returns `True` if the joint positions are close to the goals
- when taking measurements from sensors in the real world, there will always be some amount of noise
- the same is true of the joint positions reported by the gazebo simulator
- if both joint positions are within .05 radians of the goal, `True` is returned


```python
def clamp_at_boundaries(requested_j1, requested_j2):
    clamped_j1 = requested_j1
    clamped_j2 = requested_j2
```

- `clamp_at_boundaries()` is responsible for enforcing the minimum and maximum joint angles for each joint
- if the joint angles passed in are outside of the operable range, they will be “clamped” to the nearest allowable value


```python
    min_j1 = rospy.get_param('~min_joint_1_angle', 0)
    max_j1 = rospy.get_param('~max_joint_1_angle', 2*math.pi)
    min_j2 = rospy.get_param('~min_joint_2_angle', 0)
    max_j2 = rospy.get_param('~max_joint_2_angle', 2*math.pi)
```

- the minimum and maximum joint angles are retrieved from the parameter server each time `clamp_at_boundaries()` is called
- the `"~"` is the private namespace qualifier, and indicates that the parameter we wish to get is within this node’s [private namespace](http://wiki.ros.org/Names#Resolving) `/arm_mover/` (e.g. `~min_joint_1_angle` resolves to `/arm_mover/min_joint_1_angle`)
- the second parameter is the default value to be returned, in the case that `rospy.get_param()` was unable to get the parameter from the param server


```python
    if not min_j1 <= requested_j1 <= max_j1:
        clamped_j1 = min(max(requested_j1, min_j1), max_j1)
        rospy.logwarn('j1 is out of bounds, valid range (%s,%s), clamping to: %s', min_j1, max_j1, clamped_j1)

    if not min_j2 <= requested_j2 <= max_j2:
        clamped_j2 = min(max(requested_j2, min_j2), max_j2)
        rospy.logwarn('j2 is out of bounds, valid range (%s,%s), clamping to: %s', min_j2, max_j2, clamped_j2)

    return clamped_j1, clamped_j2
```

- the rest of this function simply clamps the joint angle if necessary
- warning messages are logged if the requested joint angles are out of bounds


```python
def move_arm(pos_j1, pos_j2):
    time_elapsed = rospy.Time.now()
    j1_publisher.publish(pos_j1)
    j2_publisher.publish(pos_j2)

    while True:
        joint_state = rospy.wait_for_message('/simple_arm/joint_states', JointState)
        if at_goal(joint_state.position[0], pos_j1, joint_state.position[1], pos_j2):
            time_elapsed = joint_state.header.stamp - time_elapsed
            break

    return time_elapsed
```

- `move_arm()` commands the arm to move, returning the amount of time that elapsed while the arm was moving
- within the function we are using the `rospy.wait_for_message()` call to receive `JointState` messages from the `/simple_arm/joint_states` topic
- this is blocking function call, meaning that it will not return until a message has been received on the `/simple_arm/joint_states` topic


- in general, you should not use `wait_for_message()`
- we simply use it here for the sake of clarity, and because `move_arm` is being called from the `handle_safe_move_request()` function, which demands that the response message is passed back as a return parameter


```python
def handle_safe_move_request(req):
    rospy.loginfo('GoToPositionRequest Received - j1:%s, j2:%s',
                   req.joint_1, req.joint_2)
    clamp_j1, clamp_j2 = clamp_at_boundaries(req.joint_1, req.joint_2)
    time_elapsed = move_arm(clamp_j1, clamp_j2)

    return GoToPositionResponse(time_elapsed)
```

- this is the service handler function
- when a service client sends a `GoToPosition` request message to the `safe_move` service, this function is called
- the function parameter req is of type `GoToPositionRequest`
- the service response is of type `GoToPositionResponse`
- this is the service handler function, it is called whenever a new service request is received
- the response to the service request is returned from the function


- `move_arm()` is blocking, and will not return until the arm has finished its movement
- incoming messages cannot be processed, and no other useful work can be done in the python script while the arm is performing it’s movement command
- while this poses no real problem for this example, it is a practice that should generally be avoided
- one great way to avoid blocking the thread of execution would be to use [Action](http://wiki.ros.org/actionlib)
- here’s some [informative documentation](http://wiki.ros.org/ROS/Patterns/Communication#Communication_via_Topics_vs_Services_vs_X) describing when it’s best to use a Topic versus a Service, versus an Action


```python
def mover_service():
    rospy.init_node('arm_mover')
    service = rospy.Service('~safe_move', GoToPosition, handle_safe_move_request)
    rospy.spin()
```

- here the node is initialized with the name “arm_mover”, and the `GoToPosition` service is created with the name `"safe_move"`
- as mentioned previously, the “~” qualifier identifies that `safe_move` is meant to belong to this node’s private namespace
- the resulting service name will be `/arm_mover/safe_move`
- the third parameter to the `rospy.Service()` call is the function that should be called when a service request is received
- lastly, `rospy.spin()` simply blocks until a shutdown request is received by the node
- failure to include this line would result in `mover_service()` returning, and the script completing execution


```python
if __name__ == '__main__':
    j1_publisher = rospy.Publisher('/simple_arm/joint_1_position_controller/command', Float64, queue_size=10)
    j2_publisher = rospy.Publisher('/simple_arm/joint_2_position_controller/command', Float64, queue_size=10)

    try:
        mover_service()
    except rospy.ROSInterruptException:
        pass
```

- this section of code is similar, to that of `simple_mover()`

# Arm Mover: Launch and Interact

### Launching the project with the new service

- to get the `arm_mover` node, and accompanying `safe_move` service to launch along with all of the other nodes, you will modify `robot_spawn.launch`


- launch files, when they exist, are located within the `launch` directory in the root of a catkin package
- `simple_arm`’s launch file is located in `~/catkin_ws/src/simple_arm/launch`
- to get the `arm_mover` node to launch, simply add the following:

```xml
  <!-- The arm mover node -->
  <node name="arm_mover" type="arm_mover" pkg="simple_arm">
    <rosparam>
      min_joint_1_angle: 0
      max_joint_1_angle: 1.57
      min_joint_2_angle: 0
      max_joint_2_angle: 1.0
    </rosparam>
  </node>
```

- more information on the format of the launch file can be found [here](http://wiki.ros.org/roslaunch/XML)

### Testing the new service

- now that you've modified the launch file, you are ready to test everything out
- to do so, launch the `simple_arm`, verify that the `arm_mover` node is running, and that the `safe_move` service is listed:
- you will need to make sure that you've exited out of your previous `roslaunch` session before re-launching

```shell
cd ~/catkin_ws
catkin_make
source devel/setup.bash
roslaunch simple_arm robot_spawn.launch
```

- then, in a new terminal, verify that the node and service have indeed launched

```shell
rosnode list
rosservice list
```

- assuming that both the service (`/arm_mover/safe_move`) and the node (`/arm_mover`) show up as expected (if they've not, check the logs in the `roscore` console), you can now interact with the service using `rosservice`


- to view the camera image stream, you can use the command `rqt_image_view` (you can learn more about `rqt` and the associated tools [here](http://wiki.ros.org/rqt)):

```shell
rqt_image_view /rgb_camera/image_raw
```

### Adjusting the view

- the camera is displaying a gray image
- this is as to be expected, given that it is straight up, towards the gray sky of our gazebo world


- to point the camera towards the numbered blocks on the counter top, we would need to rotate both joint 1 and joint 2 by approximately pi/2 radians

```shell
cd ~/catkin_ws/
source devel/setup.bash
rosservice call /arm_mover/safe_move "joint_1: 1.57
joint_2: 1.57"
```

- `rosservice call` can tab-complete the request message, so that you don’t have to worry about writing it out by hand
- also, be sure to include a line break between the two joint parameters


- upon entering the command, you should be able to see the arm move, and eventually stop, reporting the amount of time it took to move the arm to the console - this is as expected
- what was not expected is the resulting position of the arm
- looking at the `roscore` console, we can very clearly see what the problem was
- the requested angle for joint 2 was out of the safe bounds
- we requested 1.57 radians, but the maximum joint angle was set to 1.0 radians


- by setting the `max_joint_2_angle` on the parameter server, we should be able to bring the blocks into view the next time a service call is made
- to increase joint 2’s maximum angle, you can use the command `rosparam`

```shell
rosparam set /arm_mover/max_joint_2_angle 1.57
```

- now we should be able to move the arm such that all of the blocks are within the field of view of the camera:

```shell
rosservice call /arm_mover/safe_move "joint_1: 1.57
joint_2: 1.57"
```

- and there you have it - all of the blocks are within the field of view!

# ROS Subscribers

- a Subscriber enables your node to read messages from a topic, allowing useful data to be streamed into the node
- in Python, ROS subscribers frequently have the following format, although other parameters and arguments are possible:

```python
sub1 = rospy.Subscriber("/topic_name", message_type, callback_function)
```

- the `"/topic_name"` indicates which topic the Subscriber should listen to
- the `message_type` is the type of message being published on `"/topic_name"`
- the `callback_function` is the name of the function that should be called with each incoming message
  - each time a message is received, it is passed as an argument to `callback_function`
  - typically, this function is defined in your node to perform a useful action with the incoming data
  - note that unlike service handler functions, the `callback_function` is not required to return anything


- for more information about subscribers, see the documentation [here](http://docs.ros.org/api/rospy/html/rospy.topics.Subscriber-class.html)

- **Q:** Which of the following ROS nodes would likely need a Subscriber?
- **A:** Any node that will will require a steady stream of input data to accomplish a task will need a subscriber to get the data into the node. For example: A node for an autonomous vehicle that implements pedestrian detection using camera data. A controller node for a lunar rover which implements the actuation of the throttle and brake given target velocities as input.

# Look Away

- to see a Subscriber in action, you'll write a node called `look_away` which will subscribe to the `/rgb_camera/image_raw` topic, which has image data from the camera mounted on the end of the robotic arm
- whenever the camera is pointed towards an uninteresting image - in this case, an image with uniform color - the callback function will move the arm to something more interesting
- there are a few extra pieces in the code to ensure that this procedure is executed smoothly, but you will learn more about them later


#### Creating the empty look_away node script

- just as before when you created the `arm_mover` and `simple_mover` nodes, you can create the look_away node as follows:

```shell
cd ~/catkin_ws
cd src/simple_arm/scripts
touch look_away
chmod u+x look_away
```


#### Troubleshooting look_away

- in some cases `look_away` is executing when running it manually but is not executing automatically with `roslaunch`
- this is typically a timing issue
- if `look_away` starts before the system has fully initialized, then `look_away` hangs in the call to `safe_move`
- student *jsteinbae* offered a great solution to this issue:
  - *my workaround was to add `wait_for_message` to the `look_away` node before subscribing to the topics
  - this ensures that the callbacks are not called before the gazebo simulation (publishing these topics) is fully initialized

```python
def __init__(self):
    rospy.init_node('look_away')
    self.last_position = None
    self.arm_moving = False

    rospy.wait_for_message('/simple_arm/joint_states', JointState)
    rospy.wait_for_message('/rgb_camera/image_raw', Image)

    self.sub1 = rospy.Subscriber('/simple_arm/joint_states', 
                                 JointState, self.joint_states_callback)
    self.sub2 = rospy.Subscriber('/rgb_camera/image_raw', 
                                 Image, self.look_away_callback)
    self.safe_move = rospy.ServiceProxy('/arm_mover/safe_move', 
                                 GoToPosition)
    rospy.spin()
```


#### Updating the launch file

- just as you did with the `arm_mover` node, to get `look_away` to launch with the rest of the nodes, you will need to modify `robot_spawn.launch`, which can be found in `~/catkin_ws/src/simple_arm/launch`

```xml
  <!-- The look away node -->
  <node name="look_away" type="look_away" pkg="simple_arm"/>
```

- while editing this file, it will be helpful to set `max_joint_2_angle: 1.57` in `arm_mover` so that it isn't necessary to set it again from the command line:

```xml
  <!-- The arm mover node -->
  <node name="arm_mover" type="arm_mover" pkg="simple_arm">
    <rosparam>
      min_joint_1_angle: 0
      max_joint_1_angle: 1.57
      min_joint_2_angle: 0
      max_joint_2_angle: 1.57
    </rosparam>
  </node>
```

# Look Away: The Code

```python
#!/usr/bin/env python

import math
import rospy
from sensor_msgs.msg import Image, JointState
from simple_arm.srv import *


class LookAway(object):
    def __init__(self):
        rospy.init_node('look_away')

        self.sub1 = rospy.Subscriber('/simple_arm/joint_states', 
                                     JointState, self.joint_states_callback)
        self.sub2 = rospy.Subscriber("rgb_camera/image_raw", 
                                     Image, self.look_away_callback)
        self.safe_move = rospy.ServiceProxy('/arm_mover/safe_move', 
                                     GoToPosition)

        self.last_position = None
        self.arm_moving = False

        rospy.spin()

    def uniform_image(self, image):
        return all(value == image[0] for value in image)

    def coord_equal(self, coord_1, coord_2):
        if coord_1 is None or coord_2 is None:
            return False
        tolerance = .0005
        result = abs(coord_1[0] - coord_2[0]) <= abs(tolerance)
        result = result and abs(coord_1[1] - coord_2[1]) <= abs(tolerance)
        return result

    def joint_states_callback(self, data):
        if self.coord_equal(data.position, self.last_position):
            self.arm_moving = False
        else:
            self.last_position = data.position
            self.arm_moving = True

    def look_away_callback(self, data):
        if not self.arm_moving and self.uniform_image(data.data):
            try:
                rospy.wait_for_service('/arm_mover/safe_move')
                msg = GoToPositionRequest()
                msg.joint_1 = 1.57
                msg.joint_2 = 1.57
                response = self.safe_move(msg)

                rospy.logwarn("Camera detecting uniform image. \
                               Elapsed time to look at something nicer:\n%s", 
                               response)

            except rospy.ServiceException, e:
                rospy.logwarn("Service call failed: %s", e)



if __name__ == '__main__':
    try: 
        LookAway()
    except rospy.ROSInterruptException:
        pass
```

```python
#!/usr/bin/env python

import math
import rospy
from sensor_msgs.msg import Image, JointState
from simple_arm.srv import *
```

- the imported modules are similar to those in `simple_arm`, except this time, we have the `Image` message type being imported so that the camera data can be used


```python
class LookAway(object):
    def __init__(self):
        rospy.init_node('look_away')

        self.sub1 = rospy.Subscriber('/simple_arm/joint_states', 
                                     JointState, self.joint_states_callback)
        self.sub2 = rospy.Subscriber("rgb_camera/image_raw", 
                                     Image, self.look_away_callback)
        self.safe_move = rospy.ServiceProxy('/arm_mover/safe_move', 
                                     GoToPosition)

        self.last_position = None
        self.arm_moving = False

        rospy.spin()
```

- we define a class for this node to better keep track of the robot arm's current movement state and position history
- just as in the node definitions before, the node is initialized using `ropsy.init_node`, and at the end of the method `rospy.spin()` is used to block until a shutdown request is received by the node


- the first subscriber, `self.sub1`, subscribes to the `/simple_arm/joint_states topic`
- the node is written to check the camera only when the arm is not moving, and by subscribing to `/simple_arm/joint_states`, changes in the position of the arm can be tracked
- the message type for this topic is `JointState`, and with each message, the message data is passed to the `joint_states_callback` function


- the second subscriber, `self.sub2`, subscribes to the `/rgb_camera/image_raw topic`
- the message type here is `Image`, and with each message, the `look_away_callback` function is called


- a `ServiceProxy` is how rospy enables calling a service from a node
- the `ServiceProxy` here is created using the name of the service you wish to call along with the service class definition: in this case `/arm_mover/safe_move` and `GoToPosition`
- the actual calls to the service will take place in the `look_away_callback` method below


```python
    def uniform_image(self, image):
        return all(value == image[0] for value in image)

    def coord_equal(self, coord_1, coord_2):
        if coord_1 is None or coord_2 is None:
            return False
        tolerance = .0005
        result = abs(coord_1[0] - coord_2[0]) <= abs(tolerance)
        result = result and abs(coord_1[1] - coord_2[1]) <= abs(tolerance)
        return result
```

- there are two helper methods defined in the code: `uniform_image` and `coord_equal`
  - the `uniform_image` method takes an image as input and checks if all color values in the image are the same as the value of the first pixel
    - this essentially checks that all the color values in the image are the same
  - the `coord_equal` method returns `True` if the coordinates `coord_1` and `coord_2` have equal components up to the specified tolerance


```python
    def joint_states_callback(self, data):
        if self.coord_equal(data.position, self.last_position):
            self.arm_moving = False
        else:
            self.last_position = data.position
            self.arm_moving = True

    def look_away_callback(self, data):
        if not self.arm_moving and self.uniform_image(data.data):
            try:
                rospy.wait_for_service('/arm_mover/safe_move')
                msg = GoToPositionRequest()
                msg.joint_1 = 1.57
                msg.joint_2 = 1.57
                response = self.safe_move(msg)

                rospy.logwarn("Camera detecting uniform image. \
                               Elapsed time to look at something nicer:\n%s", 
                               response)

            except rospy.ServiceException, e:
                rospy.logwarn("Service call failed: %s", e)
```

- when `self.sub1` receives a message on `/simple_arm/joint_states` topic, the message is passed to the `joint_states_callback` in the variable `data`
- the `joint_states_callback` uses the `coord_equal` helper method to check if the current joint states provided in data are the same as the previous joint states, which are stored in `self.last_position`
- if the current and previous joint states are the same (up to the specified tolerance), then the the arm has stopped moving, so the `self.arm_moving flag` is set to `False`
- if the current and previous joint states are different, then the arm is still moving and in this case, the method updates `self.last_position` with current position data and sets `self.arm_moving` to `True`


- the `look_away_callback` is receiving data from the `/rgb_camera/image_raw` topic
- the first line of this method verifies that the arm is not moving and also checks if the the image is uniform
- if the arm isn't moving and the image is uniform, then a `GoToPositionRequest()` message is created and sent using the `safe_move` service, moving both joint angles to `1.57`
- the method also logs a message warning you that the camera has detected a uniform image along with the elapsed time to return to a nicer image

# Look Away: Launch and Interact

- you can now launch and interact with simple_arm just as before:

```shell
cd ~/catkin_ws
catkin_make
source devel/setup.bash
roslaunch simple_arm robot_spawn.launch
```

- please note that if you are having trouble with `roslaunch simple_arm robot_spawn.launch` please try the `safe_spawner.sh` script in the `scripts` folder
- you can launch by using `./safe_spawner.sh` in a terminal of your choice


- after launching, the arm should move away from the grey sky and look towards the blocks
- to view the camera image stream, you can use the same command as before:

```shell
rqt_image_view /rgb_camera/image_raw
```

- to check that everything is working as expected, open a new terminal, `source devel/setup.bash`, and send a service call to point the arm directly up towards the sky (note that the line break in the message is necessary):

```shell
rosservice call /arm_mover/safe_move "joint_1: 0
joint_2: 0"
```

# Logging

- in the code for the `simple_mover`, `arm_mover`, and `look_away` nodes, you may have noticed logging statements such as:

```shell
rospy.logwarn('j1 is out of bounds, valid range (%s,%s), clamping to: %s',
               min_j1, max_j1, clamped_j1)
```
and

```shell
rospy.loginfo('GoToPositionRequest Received - j1:%s, j2:%s', 
               req.joint_1, req.joint_2)
```


- logging statements allow ROS nodes to send messages to a log file or the console
- this allows errors and warnings to be surfaced to the user, or log data to be used later for debugging


- by default all logging messages for a node are written to the node's log file which can be found in `~/.ros/log` or `ROS_ROOT/log`
- if `roscore` is running, you can use `roscd` to find log file directory by opening a new terminal window and typing:

```shell
roscd log
```

- in this directory, you should see directories from runs of your ROS code, along with a `latest` directory with log files from the most recent run

### Logging levels and outputs

- rospy has several message levels and provides a variety of options for how to display or store these messages:

```python
rospy.logdebug(...)
rospy.loginfo(...)
rospy.logwarn(...)
rospy.logerr(...)
rospy.logfatal(...)
```

- all levels of logging messages are recorded in ROS log files, but some message levels may also be sent to Python `stdout`, Python `stderr`, or the ROS topic `/rosout`


- the `loginfo` messages are written to Python's `stdout`, while `logwarn`, `logerr`, and `logfatal` are written to Python's `stderr` by default
- additionally, `loginfo`, `logwarn`, `logerr`, and `logfatal` are written to `/rosout`
- see [here](http://wiki.ros.org/rospy/Overview/Logging) the table which summarizes the default locations log messages are written to

### Filtering and saving log messages from /rosout

- note that for messages written to `/rosout`, you can see the messages in real time as your program is running by echoing:

```shell
rostopic echo /rosout
```

- although it can be helpful to view messages this way, because of the volume of messages written to that topic, it can sometimes be helpful to filter messages by piping them to `grep`
- these grepped messages can also be saved to a file for debugging:

```shell
rostopic echo /rosout | grep insert_search_expression_here
```

```shell
rostopic echo /rosout | grep insert_search_expression_here > path_to_output/output.txt
```

### Modifying message level sent to /rosout

- although `logdebug` messages are not written to `/rosout` by default, it is possible to modify the level of logging messages written to `/rosout` to display them there, or change the level of logging messages written to `/rosout` to be more restrictive
- to do this you must set the `log_level` attribute within the `rospy.init_node` code
  - for example, if you'd like to allow lodebug messages to be written to `/rosout`, that can be done as follows:

```python
rospy.init_node('my_node', log_level=rospy.DEBUG)
```

- other possible `rospy` options for `log_level` are `INFO`, `WARN`, `ERROR`, and `FATAL`

### Modifying display of messages sent to stdout and stderr

- it is also possible to change how messages to `stdout` and `stderr` are displayed or logged
- within a package's `.launch` file, the `output` attribute for a node tag can be set to `"screen"` or `"log"`


- for example, setting `output="screen"` for the `look_away` node in `robot_spawn.launch` will display both stdout and stderr messages in the screen:

```log
  <!-- The look away node -->
  <node name="look_away" type="look_away" pkg="simple_arm" output="screen"/>
```

- if the output attribute is left empty, the default is `"log"`


- as you continue on with the project, the ROS systems that you write will become increasingly complex
- being able to use logging effectively may prove extremely useful for debugging!

# Recap

- while we’ve done our best to give you a solid understanding of the fundamental concepts of ROS, the lessons here are by no means comprehensive


- fortunately, there are a wealth of resources available online
  - [ROS Wiki](http://wiki.ros.org/)
    - this is the official source of documentation for all ROS packages
    - additionally, there are many helpful tutorials here
    - aside from a simple search on google, this is probably the first place you should go if you are having a ROS-related problem
  - [ROS Answers](http://answers.ros.org/)
    - if the wiki does not provide the answers you are looking for, ROS Answers will be the next best bet
    - with over 33,000 questions asked on ROS answers, there’s a good chance that somebody has already addressed the problem that you are dealing with
  - [ROS Cheat Sheet](https://github.com/ros/cheatsheet/releases/download/0.0.1/ROScheatsheet_catkin.pdf)
    - this is the official ROS cheat sheet
    - even though the title indicates that it is for the indigo distribution, almost all of the commands still work in Kinetic, the distribution that we are using for this program
  - [A gentle Introduction to ROS](https://cse.sc.edu/~jokane/agitr/)
    - this is a great book, and is distributed not only in paperback form, but also for free as a PDF download